In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

# Define parameters
num_points = 20
coordinates = ['x', 'y', 'z']
column_names = [f'p{point}_{coord}' for point in range(0, num_points + 0) for coord in coordinates]

# Load the data
file_path = 'data/bye_mahendra_1.txt' 
df = pd.read_csv(file_path, delimiter=r'\s+', header=None, names=column_names)

# Verify data
print("DataFrame Shape:", df.shape)  # Should be (number_of_rows, 60)
print(df.head())

# Define the connections as per the specified pairs
connections = [
    # Left Arm
    (1, 2), (2, 4), (4, 6), (6, 8),
    
    # Right Arm
    (1, 3), (3, 5), (5, 7), (7, 9),
    
    # Left Leg
    (11, 12), (12, 14), (14, 16), (16, 18),
    
    # Right Leg
    (11, 13), (13, 15), (15, 17), (17, 19),
    
    # Spine
    (0, 1), (1, 10), (10, 11)
]

# Function to get coordinates of a point given its zero-based index
def get_point_coords(row, point_index):
    point_num = point_index + 1-1  # Convert zero-based index to point number
    x = row[f'p{point_num}_x']
    y = row[f'p{point_num}_y']
    z = row[f'p{point_num}_z']
    return x, y, z

# Create frames for Plotly animation
plotly_frames = []
for frame in range(len(df)):
    row = df.iloc[frame]
    points = [get_point_coords(row, i) for i in range(num_points)]
    
    # Scatter points
    scatter = go.Scatter3d(
        x=[p[0] for p in points],
        y=[p[1] for p in points],
        z=[p[2] for p in points],
        mode='markers+text',
        marker=dict(
            size=5,
            color='blue',
            opacity=0.8
        ),
        text=[f'P{i+0}' for i in range(num_points)],
        textposition="top center",
        textfont=dict(
            size=10,
            color='black'
        )
    )
    
    # Lines for connections
    connection_traces = []
    for conn in connections:
        start, end = conn
        connection_traces.append(
            go.Scatter3d(
                x=[points[start][0], points[end][0]],
                y=[points[start][1], points[end][1]],
                z=[points[start][2], points[end][2]],
                mode='lines',
                line=dict(color='red', width=2),
                showlegend=False
            )
        )
    
    # Combine traces
    frame_data = [scatter] + connection_traces
    plotly_frames.append(go.Frame(data=frame_data, name=str(frame)))

# Initial data (first frame)
initial_row = df.iloc[0]
initial_points = [get_point_coords(initial_row, i) for i in range(num_points)]
initial_scatter = go.Scatter3d(
    x=[p[0] for p in initial_points],
    y=[p[1] for p in initial_points],
    z=[p[2] for p in initial_points],
    mode='markers+text',
    marker=dict(
        size=5,
        color='blue',
        opacity=0.8
    ),
    text=[f'P{i+0}' for i in range(num_points)],
    textposition="top center",
    textfont=dict(
        size=10,
        color='black'
    ),
    name='Points'
)

initial_connection_traces = []
for conn in connections:
    start, end = conn
    initial_connection_traces.append(
        go.Scatter3d(
            x=[initial_points[start][0], initial_points[end][0]],
            y=[initial_points[start][1], initial_points[end][1]],
            z=[initial_points[start][2], initial_points[end][2]],
            mode='lines',
            line=dict(color='red', width=2),
            showlegend=False
        )
    )

initial_data = [initial_scatter] + initial_connection_traces

# Define layout with sliders and play buttons
layout = go.Layout(
    title='Interactive 3D Animation of 20 Points with Connections',
    scene=dict(
        xaxis=dict(title='X Coordinate'),
        yaxis=dict(title='Y Coordinate'),
        zaxis=dict(title='Z Coordinate')
    ),
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(label='Play',
                     method='animate',
                     args=[None, {"frame": {"duration": 200, "redraw": True},
                                  "fromcurrent": True, "transition": {"duration": 0}}]),
                dict(label='Pause',
                     method='animate',
                     args=[[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}])
            ],
            showactive=False,
            x=0.1,
            y=0
        )
    ],
    sliders=[{
        "steps": [{
            "args": [
                [str(k)],
                {"frame": {"duration": 200, "redraw": True},
                 "mode": "immediate",
                 "transition": {"duration": 0}}
            ],
            "label": str(k + 1),
            "method": "animate"
        } for k in range(len(df))],
        "transition": {"duration": 0},
        "x": 0.1,
        "y": 0,
        "currentvalue": {"font": {"size": 12}, "prefix": "Frame: ", "visible": True, "xanchor": "center"},
        "len": 0.9
    }]
)

# Create the figure
fig_plotly = go.Figure(
    data=initial_data,
    layout=layout,
    frames=plotly_frames
)

# Display the animation
iplot(fig_plotly)
# ani.save('animation.gif', writer='imagemagick', fps=15)

DataFrame Shape: (66, 60)
       p0_x      p0_y      p0_z      p1_x      p1_y      p1_z      p2_x  \
0  0.142220  0.641839  2.370342  0.157849  0.447214  2.410516 -0.006967   
1  0.142214  0.641795  2.370238  0.157868  0.447182  2.410531 -0.006824   
2  0.142587  0.641328  2.370294  0.157891  0.447174  2.410540 -0.006814   
3  0.142772  0.641409  2.370283  0.157980  0.447160  2.410394 -0.006771   
4  0.142887  0.641397  2.370227  0.158021  0.447169  2.410386 -0.006741   

       p2_y      p2_z      p3_x  ...     p16_z     p17_x     p17_y     p17_z  \
0  0.339810  2.421211  0.322500  ...  2.617127  0.262821 -0.858402  2.590712   
1  0.339670  2.421079  0.322348  ...  2.616382  0.262760 -0.858131  2.590559   
2  0.339666  2.421060  0.322507  ...  2.616576  0.263012 -0.858500  2.590760   
3  0.339624  2.420932  0.322446  ...  2.616839  0.263553 -0.859344  2.591486   
4  0.339660  2.420904  0.323134  ...  2.616296  0.263527 -0.859238  2.591420   

      p18_x     p18_y     p18_z     p19_x 

In [5]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

# Define parameters
num_points = 20
coordinates = ['x', 'y', 'z']
column_names = [f'p{point}_{coord}' for point in range(0, num_points + 0) for coord in coordinates]

# Load the data
file_path = 'data/bye_shewta_9.txt'  
df = pd.read_csv(file_path, delimiter=r'\s+', header=None, names=column_names)

# Verify data
print("DataFrame Shape:", df.shape)  # Should be (number_of_rows, 60)
print(df.head())

# Define the connections as per the specified pairs
connections = [
    # Left Arm
    (1, 2), (2, 4), (4, 6), (6, 8),
    
    # Right Arm
    (1, 3), (3, 5), (5, 7), (7, 9),
    
    # Left Leg
    (11, 12), (12, 14), (14, 16), (16, 18),
    
    # Right Leg
    (11, 13), (13, 15), (15, 17), (17, 19),
    
    # Spine
    (0, 1), (1, 10), (10, 11)
]

# Function to get coordinates of a point given its zero-based index
def get_point_coords(row, point_index):
    point_num = point_index + 1-1  # Convert zero-based index to point number
    x = row[f'p{point_num}_x']
    y = row[f'p{point_num}_y']
    z = row[f'p{point_num}_z']
    return x, y, z

# Create frames for Plotly animation
plotly_frames = []
for frame in range(len(df)):
    row = df.iloc[frame]
    points = [get_point_coords(row, i) for i in range(num_points)]
    
    # Scatter points
    scatter = go.Scatter3d(
        x=[p[0] for p in points],
        y=[p[1] for p in points],
        z=[p[2] for p in points],
        mode='markers+text',
        marker=dict(
            size=5,
            color='blue',
            opacity=0.8
        ),
        text=[f'P{i+0}' for i in range(num_points)],
        textposition="top center",
        textfont=dict(
            size=10,
            color='black'
        )
    )
    
    # Lines for connections
    connection_traces = []
    for conn in connections:
        start, end = conn
        connection_traces.append(
            go.Scatter3d(
                x=[points[start][0], points[end][0]],
                y=[points[start][1], points[end][1]],
                z=[points[start][2], points[end][2]],
                mode='lines',
                line=dict(color='red', width=2),
                showlegend=False
            )
        )
    
    # Combine traces
    frame_data = [scatter] + connection_traces
    plotly_frames.append(go.Frame(data=frame_data, name=str(frame)))

# Initial data (first frame)
initial_row = df.iloc[0]
initial_points = [get_point_coords(initial_row, i) for i in range(num_points)]
initial_scatter = go.Scatter3d(
    x=[p[0] for p in initial_points],
    y=[p[1] for p in initial_points],
    z=[p[2] for p in initial_points],
    mode='markers+text',
    marker=dict(
        size=5,
        color='blue',
        opacity=0.8
    ),
    text=[f'P{i+0}' for i in range(num_points)],
    textposition="top center",
    textfont=dict(
        size=10,
        color='black'
    ),
    name='Points'
)

initial_connection_traces = []
for conn in connections:
    start, end = conn
    initial_connection_traces.append(
        go.Scatter3d(
            x=[initial_points[start][0], initial_points[end][0]],
            y=[initial_points[start][1], initial_points[end][1]],
            z=[initial_points[start][2], initial_points[end][2]],
            mode='lines',
            line=dict(color='red', width=2),
            showlegend=False
        )
    )

initial_data = [initial_scatter] + initial_connection_traces

# Define layout with sliders and play buttons
layout = go.Layout(
    title='Interactive 3D Animation of 20 Points with Connections',
    scene=dict(
        xaxis=dict(title='X Coordinate'),
        yaxis=dict(title='Y Coordinate'),
        zaxis=dict(title='Z Coordinate')
    ),
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(label='Play',
                     method='animate',
                     args=[None, {"frame": {"duration": 200, "redraw": True},
                                  "fromcurrent": True, "transition": {"duration": 0}}]),
                dict(label='Pause',
                     method='animate',
                     args=[[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}])
            ],
            showactive=False,
            x=0.1,
            y=0
        )
    ],
    sliders=[{
        "steps": [{
            "args": [
                [str(k)],
                {"frame": {"duration": 200, "redraw": True},
                 "mode": "immediate",
                 "transition": {"duration": 0}}
            ],
            "label": str(k + 1),
            "method": "animate"
        } for k in range(len(df))],
        "transition": {"duration": 0},
        "x": 0.1,
        "y": 0,
        "currentvalue": {"font": {"size": 12}, "prefix": "Frame: ", "visible": True, "xanchor": "center"},
        "len": 0.9
    }]
)

# Create the figure
fig_plotly = go.Figure(
    data=initial_data,
    layout=layout,
    frames=plotly_frames
)

# Display the animation
iplot(fig_plotly)
# ani.save('animation.gif', writer='imagemagick', fps=15)

DataFrame Shape: (63, 60)
       p0_x      p0_y      p0_z      p1_x      p1_y      p1_z      p2_x  \
0 -0.278453  0.542164  2.330804 -0.329087  0.390309  2.330633 -0.358655   
1 -0.278719  0.542176  2.330643 -0.329302  0.390353  2.330686 -0.358199   
2 -0.279020  0.542077  2.330177 -0.329402  0.390288  2.330639 -0.357825   
3 -0.279281  0.542092  2.330053 -0.329460  0.390299  2.330779 -0.357957   
4 -0.279544  0.542292  2.329704 -0.329652  0.390563  2.330788 -0.358055   

       p2_y      p2_z      p3_x  ...     p16_z     p17_x     p17_y     p17_z  \
0  0.268487  2.254065 -0.227166  ...  2.405362 -0.298463 -0.782065  2.565340   
1  0.268454  2.254175 -0.227503  ...  2.405523 -0.296761 -0.780010  2.567546   
2  0.267657  2.254007 -0.227626  ...  2.405362 -0.296076 -0.778474  2.569499   
3  0.267728  2.253888 -0.227738  ...  2.405829 -0.298389 -0.774910  2.593729   
4  0.267489  2.253937 -0.228011  ...  2.405905 -0.293587 -0.784439  2.582347   

      p18_x     p18_y     p18_z     p19_x 

In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

# Define parameters
num_points = 20
coordinates = ['x', 'y', 'z']
column_names = [f'p{point}_{coord}' for point in range(0, num_points + 0) for coord in coordinates]

# Load the data
file_path = 'data/bye_mahendra_9.txt'  
df = pd.read_csv(file_path, delimiter=r'\s+', header=None, names=column_names)

# Verify data
print("DataFrame Shape:", df.shape)  # Should be (number_of_rows, 60)
print(df.head())

# Define the connections as per the specified pairs
connections = [
    # Left Arm
    (1, 2), (2, 4), (4, 6), (6, 8),
    
    # Right Arm
    (1, 3), (3, 5), (5, 7), (7, 9),
    
    # Left Leg
    (11, 12), (12, 14), (14, 16), (16, 18),
    
    # Right Leg
    (11, 13), (13, 15), (15, 17), (17, 19),
    
    # Spine
    (0, 1), (1, 10), (10, 11)
]

# Function to get coordinates of a point given its zero-based index
def get_point_coords(row, point_index):
    point_num = point_index + 1-1  # Convert zero-based index to point number
    x = row[f'p{point_num}_x']
    y = row[f'p{point_num}_y']
    z = row[f'p{point_num}_z']
    return x, y, z

# Create frames for Plotly animation
plotly_frames = []
for frame in range(len(df)):
    row = df.iloc[frame]
    points = [get_point_coords(row, i) for i in range(num_points)]
    
    # Scatter points
    scatter = go.Scatter3d(
        x=[p[0] for p in points],
        y=[p[1] for p in points],
        z=[p[2] for p in points],
        mode='markers+text',
        marker=dict(
            size=5,
            color='blue',
            opacity=0.8
        ),
        text=[f'P{i+0}' for i in range(num_points)],
        textposition="top center",
        textfont=dict(
            size=10,
            color='black'
        )
    )
    
    # Lines for connections
    connection_traces = []
    for conn in connections:
        start, end = conn
        connection_traces.append(
            go.Scatter3d(
                x=[points[start][0], points[end][0]],
                y=[points[start][1], points[end][1]],
                z=[points[start][2], points[end][2]],
                mode='lines',
                line=dict(color='red', width=2),
                showlegend=False
            )
        )
    
    # Combine traces
    frame_data = [scatter] + connection_traces
    plotly_frames.append(go.Frame(data=frame_data, name=str(frame)))

# Initial data (first frame)
initial_row = df.iloc[0]
initial_points = [get_point_coords(initial_row, i) for i in range(num_points)]
initial_scatter = go.Scatter3d(
    x=[p[0] for p in initial_points],
    y=[p[1] for p in initial_points],
    z=[p[2] for p in initial_points],
    mode='markers+text',
    marker=dict(
        size=5,
        color='blue',
        opacity=0.8
    ),
    text=[f'P{i+0}' for i in range(num_points)],
    textposition="top center",
    textfont=dict(
        size=10,
        color='black'
    ),
    name='Points'
)

initial_connection_traces = []
for conn in connections:
    start, end = conn
    initial_connection_traces.append(
        go.Scatter3d(
            x=[initial_points[start][0], initial_points[end][0]],
            y=[initial_points[start][1], initial_points[end][1]],
            z=[initial_points[start][2], initial_points[end][2]],
            mode='lines',
            line=dict(color='red', width=2),
            showlegend=False
        )
    )

initial_data = [initial_scatter] + initial_connection_traces

# Define layout with sliders and play buttons
layout = go.Layout(
    title='Interactive 3D Animation of 20 Points with Connections',
    scene=dict(
        xaxis=dict(title='X Coordinate'),
        yaxis=dict(title='Y Coordinate'),
        zaxis=dict(title='Z Coordinate')
    ),
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(label='Play',
                     method='animate',
                     args=[None, {"frame": {"duration": 200, "redraw": True},
                                  "fromcurrent": True, "transition": {"duration": 0}}]),
                dict(label='Pause',
                     method='animate',
                     args=[[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}])
            ],
            showactive=False,
            x=0.1,
            y=0
        )
    ],
    sliders=[{
        "steps": [{
            "args": [
                [str(k)],
                {"frame": {"duration": 200, "redraw": True},
                 "mode": "immediate",
                 "transition": {"duration": 0}}
            ],
            "label": str(k + 1),
            "method": "animate"
        } for k in range(len(df))],
        "transition": {"duration": 0},
        "x": 0.1,
        "y": 0,
        "currentvalue": {"font": {"size": 12}, "prefix": "Frame: ", "visible": True, "xanchor": "center"},
        "len": 0.9
    }]
)

# Create the figure
fig_plotly = go.Figure(
    data=initial_data,
    layout=layout,
    frames=plotly_frames
)

# Display the animation
iplot(fig_plotly)
# ani.save('animation.gif', writer='imagemagick', fps=15)

DataFrame Shape: (60, 60)
       p0_x      p0_y      p0_z      p1_x      p1_y      p1_z      p2_x  \
0  0.103022  0.622537  2.277885  0.094512  0.448517  2.300950  0.066389   
1  0.103023  0.622544  2.277919  0.094691  0.448514  2.301092  0.066537   
2  0.103852  0.622015  2.277894  0.095338  0.448082  2.301322  0.066687   
3  0.104180  0.621980  2.277908  0.095513  0.448153  2.301372  0.066806   
4  0.104796  0.621652  2.278122  0.096141  0.447797  2.301544  0.066889   

       p2_y      p2_z      p3_x  ...     p16_z     p17_x     p17_y     p17_z  \
0  0.295478  2.191484  0.191297  ...  2.405010  0.177496 -0.871373  2.633270   
1  0.295536  2.191583  0.191476  ...  2.405018  0.177514 -0.871384  2.633278   
2  0.295552  2.191616  0.192122  ...  2.404828  0.177599 -0.871588  2.633051   
3  0.295593  2.191638  0.192298  ...  2.404723  0.177357 -0.870552  2.631596   
4  0.295610  2.191629  0.192925  ...  2.404773  0.177524 -0.870267  2.631999   

      p18_x     p18_y     p18_z     p19_x 